In [ ]:
import chipwhisperer as cw
import binascii

In [ ]:
scope = cw.scope()
target = cw.target(scope)
scope.gain.gain = 55
scope.adc.samples = 20000
scope.adc.offset = 230000 #starting offset
scope.adc.basic_mode = "rising_edge"
scope.clock.clkgen_freq = 7370000
scope.clock.adc_src = "clkgen_x1"
scope.trigger.triggers = "tio4"
scope.io.tio1 = "serial_rx"
scope.io.tio2 = "serial_tx"
scope.io.hs2 = "clkgen"

In [ ]:
prog = cw.programmers.STM32FProgrammer
fw_path = "../../hardware/victims/firmware/simpleserial-rsa/simpleserial-rsa-CWLITEARM.hex"

In [ ]:
cw.programTarget(scope, prog, fw_path)

In [ ]:
help(cw.key_text_patterns)

In [ ]:
ktp = cw.ktp.Basic(target=target)

In [ ]:
import time
text = [0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x80, 0x14, 0x00]  # manual creation of a key, text pair can be substituted here
target.key_cmd = None
target.output_cmd = None
def get_go_cmd(text):
    go_cmd = "p"
    for num in text:
        go_cmd += "{:02X}".format(num)
    go_cmd += "\\n"
    return go_cmd

target.go_cmd = get_go_cmd(text)

def run_multi_trace(scope, target, start, step, num_step):
    scope.adc.samples = step
    num_stepped = 0
    trace = np.zeros(step*num_step)
    while num_stepped < num_step:

        scope.adc.offset = start + (step * num_stepped)

        # run aux stuff that should run before the scope arms here
        scope.arm()

        # run aux stuff that should run after the scope arms here

        target.go()
        timeout = 500
        # wait for target to finish
        while target.isDone() is False:
            timeout -= 1
            time.sleep(0.01)

        try:
            ret = scope.capture()
            if ret:
                print('Timeout happened during acquisition')
        except IOError as e:
            print('IOError: %s' % str(e))

        # run aux stuff that should happen after trace here

        #_ = target.readOutput() # throw out the target response
        # append the new trace
        new_trace = scope.getLastTrace()
        if len(new_trace) < step:
            new_trace = np.zeros(step)
        trace[((num_stepped) * step):((num_stepped+1) * step)] = new_trace
        num_stepped += 1
    return trace



In [ ]:
cw.programTarget(scope, prog, fw_path)
import numpy as np
scope.adc.timeout = 200
start = 0
step = 20000
num_step = 1
end = start + (step * num_step)
next_trace = run_multi_trace(scope, target, start, step, num_step)
print(target.go_cmd)


In [ ]:
print(scope.adc.trig_count)
help(scope.adc)

In [ ]:
from bokeh.plotting import figure, show
from bokeh.io import output_notebook


output_notebook()


In [ ]:
xdata = range(len(next_trace))
p = figure()
p.line(xdata, next_trace, line_color="red")
show(p)